# Read all item IDs

In [1]:
import html

items_w_meta = set()
# skip_mag_sub = 0

with open('/home/shiv/Downloads/All_Amazon_Meta.json', 'r') as f: # 15,023,059 lines!
    line_cnt = 0
    for line in f:
        line_new = eval(line)
        
        items_w_meta.add(line_new['asin']) # collect all the item IDs
        line_cnt += 1
        if line_cnt % 100_000 == 0:
            if line_cnt % 1_000_000 == 0:
                print('!', end = '')
            else:
                print('*', end = '')  
print()
print("Total items", line_cnt)

# NOTES: lots of duplicate asins in meta file: only 14741571 items from 15023059 lines

*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!
Total items 15023059


In [2]:
import pickle
items_pkl_path = f'/home/shiv/Documents/DataScience/Capstone/Data/items_w_meta.pkl'

with open(items_pkl_path, 'wb') as handle:
    pickle.dump(items_w_meta, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
import pickle
items_pkl_path = f'/home/shiv/Documents/DataScience/Capstone/Data/items_w_meta.pkl'
with open(items_pkl_path, 'rb') as handle:
    items_w_meta = pickle.load(handle)
print(len(items_w_meta))

14741571


# Read all reviews (157M + )

1. <s>save only reviews on or after the CUTOFF_DATE</s>
2. save only verified reviews
3. only for those products for which we have the meta data

In [3]:
import json
import pandas as pd

# CUTOFF_DATE = pd.to_datetime("2017-06-01")

ratings_items = []
line_cnt = 0
with open('/home/shiv/Documents/DataScience/Capstone/Data/All_Amazon_Review_5.json', 'r') as f:
    for line in f:
        review = json.loads(line) # overall, verified, reviewerID, asin, unixReviewTime, reviewText, summary
#         ts = pd.to_datetime(review['unixReviewTime'], unit='s')
#         if ts >= CUTOFF_DATE and review['verified'] and review['asin'] in items_w_meta:
        if review['verified'] and review['asin'] in items_w_meta:
            ratings_items.append([review['reviewerID'], review['asin'],
                                  review['overall'], review['unixReviewTime']])
      
        line_cnt += 1
        if line_cnt % 100_000 == 0:
            if line_cnt % 1_000_000 == 0:
                print('!', end = '')
            else:
                print('*', end = '')
print()
print(line_cnt) # 157260920
print(len(ratings_items))

# NOTES: even though it is curated, there aren't 5 reviews per product nor 5 reviews per user
# So, choosing to only look at verified reviews and for products that have the meta information.
# Down to 133M+ reviews

*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!

In [5]:
ratings_df = pd.DataFrame(ratings_items, columns = ['reviewerID', 'asin', 
                                                    'rating', 'unixReviewTime'])
ratings_df.head()

,reviewerID,asin,rating,unixReviewTime
0,AEUZVK0XRMI40,B000HA3CEY,5.0,1363824000
1,A1U4R8DP7MLOYP,B000HA3CEY,2.0,1354060800
2,A1JIXKFDTEC6DZ,B000HA3CEY,5.0,1225411200
3,A11AP8Z9IF3NH2,B001EJZNXK,5.0,1361836800
4,A2YJWVLJ21CUFG,B001EJZNXK,4.0,1357171200


In [7]:
ratings_csv_path = f'/home/shiv/Documents/DataScience/Capstone/Data/all_reviews.csv'
ratings_df.to_csv(ratings_csv_path, header=False, index=False)

# Restart the kernel

In [1]:
import pandas as pd
ratings_csv_path = f'/home/shiv/Documents/DataScience/Capstone/Data/all_reviews_5.csv'
ratings_df = pd.read_csv(ratings_csv_path, header=None)
ratings_df.columns = ['reviewerID', 'asin', 'rating', 'unixReviewTime']

In [2]:
def get_min_reviews_prod_per_user(ratings_df, ret_min_prod, ret_min_user):
    prod_df = ratings_df[['reviewerID','asin']].groupby(['asin']).count()
    min_prod = -1
    if ret_min_prod:
        min_prod = prod_df['reviewerID'].min()
        print("min reviews per item", min_prod) #, prod_df['reviewerID'].max())
    user_df = ratings_df[['reviewerID','asin']].groupby(['reviewerID']).count()
    min_user = -1
    if ret_min_user:
        min_user = user_df['asin'].min()
        print("min reviews per user", min_user) #, user_df['asin'].max())
    return min_prod, prod_df, min_user, user_df

min_prod, prod_df, min_user, user_df = get_min_reviews_prod_per_user(ratings_df, True, True)

min reviews per item 5
min reviews per user 5


# Remove items with less than MIN_REVIEWS reviews

This is a multi-round process. In each round, we do the following:

1. Filter the ratings dataframe by removing products that have less than MIN_REVIEWS reviews
2. Next, check if the remaining users have MIN_REVIEWS_PER_USER reviews
3. If both MIN_REVIEWS and MIN_REVIEWS_PER_USER conditions are satisfied, we are done!
4. Otherwise filter the ratings dataframe by removing users that have less than MIN_REVIEWS_PER_USER reviews
5. Next, check if the remaining items have MIN_REVIEWS reviews
6. If both MIN_REVIEWS and MIN_REVIEWS_PER_USER conditions are satisfied, we are done!

In [3]:
MIN_REVIEWS = 20
MIN_REVIEWS_PER_USER = 20

print("# users", ratings_df['reviewerID'].nunique(), "#items", ratings_df['asin'].nunique())

# users 8534835 #items 3214887


In [4]:
round = 1
while True:
    print("Round #", round)
    
    prod_list = list(prod_df[prod_df['reviewerID'] >= MIN_REVIEWS].index)
    print("# items:", len(prod_list))
    
    ratings_df = ratings_df[ratings_df['asin'].isin(prod_list)]
    print(ratings_df.shape)
    
    min_prod, prod_df, min_user, user_df = get_min_reviews_prod_per_user(ratings_df, False, True)
    min_prod = MIN_REVIEWS
    
    if min_prod >= MIN_REVIEWS and min_user >= MIN_REVIEWS_PER_USER:
        break    

    user_list = list(user_df[user_df['asin'] >= MIN_REVIEWS_PER_USER].index)
    print("# users", len(user_list))
    
    ratings_df = ratings_df[ratings_df['reviewerID'].isin(user_list)]
    print(ratings_df.shape)
    
    min_prod, prod_df, min_user, user_df = get_min_reviews_prod_per_user(ratings_df, True, False)
    min_user = MIN_REVIEWS_PER_USER
    
    if min_prod >= MIN_REVIEWS and min_user >= MIN_REVIEWS_PER_USER:
        break    
    round += 1

Round # 1
# items: 1070664
(108501158, 4)
min reviews per user 1
# users 1284496
(50858773, 4)
min reviews per item 1
Round # 2
# items: 552846
(44457426, 4)
min reviews per user 1
# users 1040451
(40210494, 4)
min reviews per item 2
Round # 3
# items: 497273
(39222851, 4)
min reviews per user 3
# users 999330
(38455791, 4)
min reviews per item 5
Round # 4
# items: 486864
(38261068, 4)
min reviews per user 14
# users 990998
(38103482, 4)
min reviews per item 15
Round # 5
# items: 484648
(38061565, 4)
min reviews per user 17
# users 989235
(38028150, 4)
min reviews per item 16
Round # 6
# items: 484197
(38019593, 4)
min reviews per user 18
# users 988826
(38011832, 4)
min reviews per item 16
Round # 7
# items: 484088
(38009765, 4)
min reviews per user 19
# users 988743
(38008188, 4)
min reviews per item 19
Round # 8
# items: 484066
(38007770, 4)
min reviews per user 19
# users 988722
(38007371, 4)
min reviews per item 19
Round # 9
# items: 484062
(38007295, 4)
min reviews per user 19
# 

In [5]:
ratings_df.to_csv(f'/home/shiv/Documents/DataScience/Capstone/Data/all_reviews_{MIN_REVIEWS}.csv', 
                  header=False, index=False)

# Get the meta information for the products in all_reviews_{MIN_REVIEWS}

In [6]:
# import pandas as pd
# ratings_df = pd.read_csv(f'/home/shiv/Documents/DataScience/Capstone/Data/all_reviews_{MIN_REVIEWS}.csv',
#                          header=None)
# ratings_df.columns = ['reviewerID', 'asin', 'rating', 'unixReviewTime']
print(ratings_df.shape)
ratings_df

(38007219, 4)


,reviewerID,asin,rating,unixReviewTime
5,A14E6RS86U3IWJ,B001EJY2IC,5.0,1363737600
6,A279VRO0YVM43V,B001EJY2IC,4.0,1362355200
9,A1HJ4BHRNFBGSX,B001EJY2IC,5.0,1473033600
11,A139FFBSZTFUAT,B001EJY2IC,5.0,1468281600
12,A2MBPLWN8MKY93,B001EJY2IC,5.0,1466208000
...,...,...,...,...
128135433,A1WAJWOFBE4PPR,B01HB9Q7CW,5.0,1537747200
128135642,A1MCN1E5GNFNXJ,B01HI9W5HQ,1.0,1538006400
128135643,A3IAV917ZH50SE,B01HI9W5HQ,4.0,1535068800
128135644,A2RPMPBKX1MWR7,B01HI9W5HQ,5.0,1534550400


In [7]:
ratings_df.reset_index(inplace=True,drop=True)

set_items = set(ratings_df['asin'].unique()) # all unique items in the ratings dataframe
print("# users", ratings_df['reviewerID'].nunique(), "# items", len(set_items))

# users 988718 # items 484062


## Clean up meta information

Meta information consists of the following:

1. main category
2. categories
3. price
4. title

- Each of the columns can come with HTML escape sequences that need to be removed.
- Each of the columns can be empty!
- price and title columns are very noisy!
- There are some items that have too many categories. Only the first 5 are saved to conserve space

In [8]:
import html

items = []
meta_items = set()

with open('/home/shiv/Downloads/All_Amazon_Meta.json', 'r') as f: # 15,023,059 lines!
    line_cnt = 0
    empty_main_cat = 0
    empty_lst_cat = 0
    empty_price = 0
    empty_title = 0
    missing_price_cnt = 0
    missing_title_cnt = 0
    dups_cnt = 0
    
    for line in f:
        line_new = eval(line)

        asin = line_new['asin']
        if asin in set_items:
            lst_vals = []
            # asin
            lst_vals.append(line_new['asin'])

            # main_cat
            main_cat = line_new['main_cat']
            main_cat = html.unescape(main_cat)
            lst_vals.append(main_cat)
            if line_new['main_cat'] == '':
                empty_main_cat += 1
            
            # categories
            lst_categories = line_new['category']
            if len(lst_categories) > 5:
                lst_categories = lst_categories[:5]
            cats = []
            for cat in lst_categories:
                cats.append(html.unescape(cat))
            if len(cats) == 0:
                empty_lst_cat += 1
            lst_vals.append('|'.join(cats))
            
            # price
            if 'price' in line_new:
                lst_vals.append(line_new['price'])
                if line_new['price'].strip() == '':
                    empty_price += 1
            else:
                missing_price_cnt += 1
                lst_vals.append('')

            # title    
            if 'title' in line_new:
                title = line_new['title'].strip()
                title = html.unescape(title)
                if title == '':
                    empty_title += 1
                    title = 'Unknown'
                lst_vals.append(title)                
            else:
                missing_title_cnt += 1
                lst_vals.append('')

            if asin in meta_items:
                lst_vals.append(1)
                dups_cnt += 1
            else:
                lst_vals.append(0)
                meta_items.add(asin)
                
            items.append(lst_vals)

        line_cnt += 1
        if line_cnt % 100_000 == 0:
            if line_cnt % 1_000_000 == 0:
                print('!', end = '')
            else:
                print('*', end = '')        

print()
print(len(items), "duplicates", dups_cnt)
print("main category", empty_main_cat)
print("lst category", empty_lst_cat)
print("title", missing_title_cnt, empty_title)
print("price", missing_price_cnt, empty_price)

*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!*********!
498489 duplicates 14427
main category 1269
lst category 19596
title 0 104
price 0 129512


In [9]:
# Assert if we did not get the meta data for all unique items
lst_no_meta_items = list(set_items - meta_items)
assert(len(lst_no_meta_items) == 0)
# ratings_df = ratings_df[~ratings_df['asin'].isin(lst_no_meta_items)]

In [10]:
items_df = pd.DataFrame(items, columns = ['asin', 'main_cat', 'category', 'price', 'title', 'duplicate'])
# sorted(items_df['main_cat'].unique())

## Fix html tags appearing in main_cat, use alt instead

In [12]:
from html.parser import HTMLParser
class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.found_tag = False
        self.recorded_alt = False
        self.alt_cat = ''
        
    def handle_starttag(self, tag, attrs):
        # print("Encountered a start tag:", attrs)
        if tag == 'img':
            self.found_tag = True
            for a in attrs:
                if a[0] == 'alt':
                    self.recorded_alt = True
                    self.alt_cat = a[1].title()

    def handle_endtag(self, tag):
        # print("Encountered an end tag :", tag)
        self.found_tag = False

    def handle_data(self, data):
        # print("Encountered some data  :", data)
        pass

main_cat_dict = {}
parser = MyHTMLParser()
lst_main_cat = items_df.main_cat.unique()

for cat in lst_main_cat:
    parser.recorded_alt = False
    parser.feed(cat)
    if parser.recorded_alt:
        # print(parser.alt_cat)
        main_cat_dict[cat] = parser.alt_cat
    else:
        main_cat_dict[cat] = cat

In [13]:
items_df['main_cat_tr'] = items_df['main_cat'].apply(lambda x: main_cat_dict[x])
# sorted(items_df['main_cat_tr'].unique())

## Pick first category if main_cat_tr is empty

In [14]:
items_df['main_cat_tr'] = items_df[['main_cat_tr','category']].apply(lambda x: x['category'].split('|')[0] if x['main_cat_tr'] == '' else x['main_cat_tr'], axis=1)

## Consolidate main category

- Some of the main categories have very few items. We fold them into an overarching category by looking at the categories list (if not empty) and/or the title.

In [15]:
print(len(sorted(items_df['main_cat_tr'].unique())))
items_df[['main_cat_tr', 'main_cat']].groupby(['main_cat_tr']).count()

51


,main_cat
main_cat_tr,
All Beauty,1513
All Electronics,10417
Amazon Devices,668
Amazon Fashion,55759
Amazon Fire TV,5
Amazon Home,68701
Apple Products,43
Appliances,249
"Arts, Crafts & Sewing",7018


In [20]:
main_cat_map = { # Transform the main_cat if the count for a main category is low
    'MEMBERSHIPS & SUBSCRIPTIONS': 'Home & Kitchen',
    'Entertainment': 'Collectibles',
    'Amazon Fire TV': 'Amazon Devices',
    'Fire Phone' : 'Amazon Devices',
    'Sports Collectibles': 'Collectibles',
    'Collectibles & Fine Art' : 'Collectibles',
    'Collectible Coins': 'Collectibles',
    'Buy a Kindle': 'Kindle',
    'Kindle Store': 'Kindle'
}

In [21]:
items_df['main_cat_tr'] = items_df['main_cat_tr'].apply(lambda x: main_cat_map[x] if x in main_cat_map else x)

In [22]:
len(sorted(items_df['main_cat_tr'].unique()))

45

## Fix category lists:
1. remove less frequently occuring categories
2. remove main_category from this list of (sub)categories

In [23]:
from collections import defaultdict
from tqdm import tqdm

cat_count = defaultdict(lambda: 0)
with tqdm(total=items_df.shape[0]) as pbar:
    categories = items_df['category'].values
    for row in categories:
        for cat in row.split('|'):
            cat_count[cat] += 1
        pbar.update(1)

100%|█████████████████████████████████████████████████████████████████████████████████████| 498489/498489 [00:00<00:00, 1533226.08it/s]


In [24]:
items_df['category_tr'] = ''
def update_category(row):
    categories = row['category']
    new_cat = []
    # print(categories)
    for cat in categories.split('|'):
        if cat_count[cat] >= 5 and cat != row['main_cat_tr']:
            new_cat.append(cat)
    if len(new_cat) > 0:
        row['category_tr'] = '|'.join(new_cat)
    else:
        row['category_tr'] = ''
    return row
items_df = items_df.apply(update_category, axis=1)

In [25]:
# Ensure that the duplicate rows have no new information
tmp_df = items_df[items_df['duplicate']==1]
with tqdm(total=tmp_df.shape[0]) as pbar:
    for _, row in tmp_df.iterrows():
        dup_rows = items_df[items_df['asin']==row['asin']]
        assert(dup_rows.shape[0]==2) # only 2 rows share an asin
        assert(dup_rows.iloc[0]['category'] == dup_rows.iloc[1]['category'])
        assert(dup_rows.iloc[0]['main_cat_tr'] == dup_rows.iloc[1]['main_cat_tr'])
        pbar.update(1)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 14427/14427 [08:23<00:00, 28.67it/s]


## Drop the unnecessary columns; save all_meta_{MIN_REVIEWS}

In [26]:
items_df.drop(columns=['main_cat','category','duplicate'], inplace=True)
print(items_df.shape)
items_df.drop_duplicates(inplace=True)
print(items_df.shape)
items_df.head()

(498489, 5)
(484062, 5)


,asin,price,title,main_cat_tr,category_tr
0,7709260373,$23.99 - $29.99,LJYH Children's Collar Motorcycle Faux Leather...,Amazon Fashion,"Clothing, Shoes & Jewelry|Boys|Clothing|Jacket..."
1,9792252916,,"Casio Men's Quartz Resin Casual Watch, Color:B...",Amazon Fashion,"Clothing, Shoes & Jewelry|Men|Watches|Wrist Wa..."
2,B00001QHXX,,Politically Incorrect Nixon Vinyl Mask,Amazon Fashion,"Clothing, Shoes & Jewelry|Costumes & Accessori..."
3,B00001TOXD,$4.99,Adult Witch Broom,Toys & Games,"Clothing, Shoes & Jewelry|Costumes & Accessori..."
4,B00001T38Y,$24.98,Star Wars Stormtrooper Deluxe Adult Mask,Amazon Fashion,"Clothing, Shoes & Jewelry|Costumes & Accessori..."


In [27]:
items_df.rename(columns={'main_cat_tr': 'main_cat', 'category_tr': 'category'}, inplace=True)

In [28]:
items_df.to_csv(f'/home/shiv/Documents/DataScience/Capstone/Data/all_meta_{MIN_REVIEWS}.csv', 
                header=False, index=False)

# Prepare for wide_deep

1. Restart the kernel
2. Read the ratings and items dataframes
3. Prepare Electronics, Home and Games datasets

In [2]:
import pandas as pd
MIN_REVIEWS = 20

ratings_df = pd.read_csv(f'/home/shiv/Documents/DataScience/Capstone/Data/all_reviews_{MIN_REVIEWS}.csv', 
                         header=None)
ratings_df.columns = ['reviewerID', 'asin', 'rating', 'unixTimeStamp']     

In [3]:
items_df = pd.read_csv(f'/home/shiv/Documents/DataScience/Capstone/Data/all_meta_{MIN_REVIEWS}.csv', 
                       header=None)
# items_df.head()

In [4]:
items_df.columns=['asin','price','title','main_cat','category']
items_df['category'].fillna('', inplace=True)
items_df['price'].fillna('$$$', inplace=True)

In [89]:
# Sample to show how to drop a main category

# mag_subs = items_df[items_df['main_cat']=='Magazine Subscriptions']['asin'].values
# ratings_df=ratings_df[~ratings_df['asin'].isin(mag_subs)]
# items_df=items_df[~items_df['asin'].isin(mag_subs)]
# ratings_df.reset_index(inplace=True,drop=True)
# items_df.reset_index(inplace=True,drop=True)

In [5]:
len(sorted(items_df['main_cat'].unique()))

45

# Prepare Electronics, Home, Games datasets

Steps:

1. Filter the items dataframe to remove all items whose main_cat is not in the list of main_categories for each umbrella category (e.g. Electronics)
2. Filter the rating dataframe to remove all reviews that do not belong to the list of item IDs left in the items dataframe after step 1.
3. Convert hexadecimal reviewerID in the ratings dataframe to userID (0 based)
4. Save the reviewerID dict in a pkl file
5. Left merge ratings and items filtered dataframe on asin (hexadecimal)
6. Convert hexadecimal asin to itemID (0 based)
7. Save the itemID dict (for UI) in a pkl file
8. Add genre; currently, consider the main categories or if an item in category matches the sub category list
9. Save the prepared dataset for use by modeling

In [21]:
import pickle
DATA_DIR = '/home/shiv/Documents/DataScience/Capstone/Data/'

def prepare_dataset(lst_main_cat, lst_sub_cat, items_df, ratings_df, dataset):
    # Step 1: filter the items dataframe
    items_filtered_df = items_df[items_df['main_cat'].isin(lst_main_cat)].copy()
    items_filtered_ids = items_filtered_df['asin'].values

    # Step 2: filter the ratings dataframe
    ratings_filtered_df = ratings_df[ratings_df['asin'].isin(items_filtered_ids)].copy()
    
    print(dataset)
    print("Num ratings:", ratings_filtered_df.shape[0])
    print("users", ratings_filtered_df['reviewerID'].nunique(), "items", ratings_filtered_df['asin'].nunique())
    
    # Step 3: convert reviewerID to userID (0 based)
    reviewers_dict = {}
    reviewer_id = 0
    for _, row in ratings_filtered_df.iterrows():
        if row['reviewerID'] not in reviewers_dict:
            reviewers_dict[row['reviewerID']] = reviewer_id
            reviewer_id += 1
            
    ratings_filtered_df['userID'] = ratings_filtered_df['reviewerID'].apply(lambda x: reviewers_dict[x])

    # Step 4: save the reviewerID dict (perhaps for UI)
    users_pkl_path = DATA_DIR + f'users_{dataset[0].lower()}_{MIN_REVIEWS}.pkl'

    with open(users_pkl_path, 'wb') as handle:
        pickle.dump(reviewers_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

    ratings_filtered_df.drop(columns=['reviewerID'], inplace=True) # we will henceforth use userID

    # Step 5: left merge the ratings with items on asin
    data = ratings_filtered_df.merge(items_filtered_df, on=['asin'], how='left')
    print("merged dataframe", data.shape)
    data['category'] = data['category'].astype('string')
    
    # Step 6: convert asin to itemID (0 based)
    items_dict = {}
    item_id = 0

    for _, row in data.iterrows():
        if row['asin'] not in items_dict:
            items_dict[row['asin']] = item_id
            item_id += 1
    data['itemID'] = data['asin'].apply(lambda x: items_dict[x])
    
    # Step 7: save the itemID dict (for UI)
    items_pkl_path = DATA_DIR + f'items_{dataset[0].lower()}_{MIN_REVIEWS}.pkl'

    with open(items_pkl_path, 'wb') as handle:
        pickle.dump(items_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)        
    data.drop(columns=['asin'], inplace=True) # we will henceforth use itemID
    
    # Step 8: add genre; 
    # currently, consider the main categories or if an item in category matches the sub category list
    data['genre'] = ''

    def update_category(row):
        categories = row['category']
        new_cat = []
        new_cat.append(row['main_cat'])
        # print(categories)
        for cat in categories.split('|'):
            if cat.strip() != '' and ((cat in lst_main_cat) or (cat in lst_sub_cat)):
                new_cat.append(cat)
        row['genre'] = '|'.join(new_cat)
        return row
    data = data.apply(update_category, axis=1)
    
    # Step 9: save the prepared dataset to be used by wide_deep model
    # data.drop(columns=['category'], inplace=True)
    data.reset_index(inplace=True, drop=True)
    data = data[['userID','itemID', 'rating','genre','unixTimeStamp','title','price','main_cat','category']]
    data.to_csv(DATA_DIR + f'wide_deep_amzn_{dataset[0].lower()}_{MIN_REVIEWS}.csv', 
                 header=False, index=False)
    
    del items_filtered_df
    del ratings_filtered_df
    del data

### Electronics

In [22]:
# main_cat = 'Electronics'
lst_main_cat = ['All Electronics', 'Amazon Devices', 'Apple Products', 
                 'Camera & Photo', 'Car Electronics', 'Cell Phones & Accessories', 'Computers',
                 'Electronics', 'GPS & Navigation', 'Home Audio & Theater', 'Industrial & Scientific',
                 'Portable Audio & Accessories']

lst_sub_cat = ['Accessories','Computers & Accessories','Office Products','Video Games',
               'Accessories & Supplies','Tools & Home Improvement','Computer Accessories & Peripherals',
               'Audio & Video Accessories', 'Automotive', 'Office & School Supplies',
               'Car & Vehicle Electronics', 'Industrial & Scientific','Sports & Outdoors','Office Electronics',
               'Home & Kitchen','Musical Instruments','Portable Audio & Video','Electrical',
               'Clothing, Shoes & Jewelry','Toys & Games','Laptop Accessories','Home Audio',
               'Controllers','Computer Components','Sports & Fitness']
prepare_dataset(lst_main_cat, lst_sub_cat, items_df, ratings_df, "Electronics")

Electronics
Num ratings: 5613183
users 830668 items 63725
merged dataframe (5613183, 8)


### Home

In [24]:
# main_cat = 'Home'
lst_main_cat = ['Amazon Home', 'Appliances', 'Home & Kitchen', 
                'Patio, Lawn & Garden', 'Tools & Home Improvement']

lst_sub_cat = ['Kitchen & Dining', 'Industrial & Scientific', 'Power & Hand Tools',
               'Automotive', 'Arts, Crafts & Sewing', 'Office Products',
               'Electronics', 'Sports & Outdoors', 'Home Dcor', 'Accessories', 'Hand Tools',
               'Office & School Supplies', 'Gardening & Lawn Care',
               'Hardware', 'Storage & Organization', 'Lighting & Ceiling Fans',
               'Kitchen Utensils & Gadgets', 'Electrical', 'Furniture', 'Pet Supplies',
               'Building Supplies', 'Bedding', 'Sports & Fitness', 'Safety & Security',
               'Outdoor Recreation', 'Power Tool Parts & Accessories', 'Kitchen & Bath Fixtures',
               'Parts & Accessories', 'Small Appliances', 'Replacement Parts',
               'Crafting', 'Sewing', 'Tools & Equipment', 'Outdoor Dcor', 'Patio Furniture & Accessories',
               'Grills & Outdoor Cooking', 'Power Tools', 'Rough Plumbing', 'Bath',
               'Bakeware', 'Accessories & Supplies', 'Heating, Cooling & Air Quality',
               'Outdoor Power Tools', 'Outdoor Lighting', 'Paint, Wall Treatments & Supplies',
               'Home Dcor Accents', 'Pools, Hot Tubs & Supplies', 'Bathroom Fixtures',
               'Test, Measure & Inspect', 'Bathroom Accessories', 'Personal Protective Equipment',
               'Fasteners', 'Vacuums & Floor Care', 'Clothing & Closet Storage', 'Exterior Accessories',
               'Replacement Parts & Accessories', 'Desk Accessories & Workspace Organizers',
               'Outdoor Cooking Tools & Accessories', 'HVAC']
prepare_dataset(lst_main_cat, lst_sub_cat, items_df, ratings_df, "Home")

Home
Num ratings: 7575829
users 882963 items 92562
merged dataframe (7575829, 8)


### Games

In [29]:
# main_cat = 'Games'
lst_main_cat = ['Sports & Outdoors', 'Toys & Games', 'Video Games']
lst_sub_cat = ['Sports & Fitness', 'Accessories', 'Clothing, Shoes & Jewelry', 'Home & Kitchen',
               'Outdoor Recreation', 'Clothing', 'Electronics', 'Games', 
               'Sports & Outdoor Play', 'Cycling', 'Exercise & Fitness', 'Camping & Hiking',
               'Leisure Sports & Game Room', 'Hunting & Fishing', 'Retro Gaming & Microconsoles',
               'Costumes & Accessories', 'Dress Up & Pretend Play', 'Shoes', 'Golf', 'Hobbies',
               'Water Sports', 'Controllers', 'Xbox One', 'PlayStation 3', 'Motorcycle & Powersports',
               'Arts & Crafts', 'Replacement Parts', 'Xbox 360', 'Other Sports', 'Crafting',
               'Wii', 'Learning & Education', 'Sports', 'Active', 'Consoles',
               'Painting, Drawing & Art Supplies', 'Audio & Video Accessories', 'Nintendo 3DS & 2DS',
               'Athletic', 'Skates, Skateboards & Scooters', 'Building Toys', 'Building Sets']
prepare_dataset(lst_main_cat, lst_sub_cat, items_df, ratings_df, "Games")

Games
Num ratings: 3735507
users 748773 items 56219
merged dataframe (3735507, 8)


# Code to discover relevant sub categories to build the lst_sub_cat

In [ ]:
from collections import defaultdict

ratings_g_df = pd.read_csv(DATA_DIR + 'wide_deep_amzn_g_20.csv', header=None, low_memory=False)
ratings_g_df.columns=['userID','itemID', 'rating','genre','unixTimeStamp','title','price','main_cat','category']
ratings_g_df['category'].fillna('', inplace=True)
ratings_g_df['price'].fillna('$$$', inplace=True)
    
lst = list(ratings_g_df['category'].unique())
word_count = defaultdict(int)
for l in lst:
    for w in l.split('|'):
        word_count[w] += 1
dict(sorted(word_count.items(), key=lambda item: item[1], reverse=True))

# Additional datasets: Books, Health & Personal Care, Fashion

In [ ]:
# main_cat = 'Books'
lst_main_cat = ['Books', 'Kindle', 'Audible audiobooks']

In [ ]:
# main_cat = 'Health & Personal Care'
lst_main_cat = ['All Beauty', 'Health & Personal Care', 'Luxury Beauty']

# main_cat = 'Fashion'
lst_main_cat = ['Amazon Fashion', 'Clothing, Shoes & Jewelry']